In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

# Data

In [29]:
# Load the dataframes
train_data = pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

# Preview the dataframes
display(train_data.head(10))
display(test_data.head(10))

,Sequence,Active
0,DKWL,0
1,FCHN,0
2,KDQP,0
3,FNWI,0
4,NKRM,0
5,IWHA,0
6,GEFV,0
7,KARS,0
8,IMGW,0
9,KYRK,0


,Sequence
0,HWFK
1,MWPW
2,ALDV
3,NTLG
4,LHYY
5,AFGM
6,TPNY
7,EAKD
8,NDKE
9,GFIS


,Active
count,112000.000000
mean,0.037616
std,0.190267
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [3]:
def create_char_df(df):
    """Create a dataframe that contains four columns (one for each char in 'Sequence')."""

    # Split sequences into characters
    first = []
    second = []
    third = []
    fourth = []

    for _, row in df.iterrows():
        ch_1, ch_2, ch_3, ch_4 = row["Sequence"]

        first.append(ch_1)
        second.append(ch_2)
        third.append(ch_3)
        fourth.append(ch_4)

    df_dict = {
        "first": first,
        "second": second,
        "third": third,
        "fourth": fourth
    }
    return pd.DataFrame(df_dict)

X_df = create_char_df(train_data)
X_test_df = create_char_df(test_data)

display(X_df.head(5))
display(X_test_df.head(5))

,first,second,third,fourth
0,D,K,W,L
1,F,C,H,N
2,K,D,Q,P
3,F,N,W,I
4,N,K,R,M


,first,second,third,fourth
0,H,W,F,K
1,M,W,P,W
2,A,L,D,V
3,N,T,L,G
4,L,H,Y,Y


## OneHot Encoding

In [4]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_df)

display(enc.categories_)

# Ensure the onehots are the same for each letter
assert((enc.categories_[0] == enc.categories_[1]).all() and (enc.categories_[1] == enc.categories_[2]).all() and (enc.categories_[2] == enc.categories_[3]).all())

[array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object),
 array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
        'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)]

# Training

In [23]:
from sklearn.model_selection import train_test_split

X = enc.transform(X_df).toarray().reshape(-1, 4, 20)
y = train_data["Active"].values

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.05, random_state=1337)

# Create training np arrays
#X_train = enc.transform(X_df).toarray().reshape(-1, 4, 20)
#y_train = train_data["Active"].values

In [46]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [56]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

#clf = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(15,), batch_size=128, verbose=True), verbose=True)
#clf.fit(X_train, y_train)

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional

batch_size = 128

model_lstm = Sequential()
model_lstm.add(
    Bidirectional(LSTM(20, activation='relu', input_shape=(4, 20)))
)
model_lstm.add(Dense(1, activation = 'sigmoid'))
model_lstm.compile(
    loss='binary_crossentropy',
    optimizer='Adam',
    metrics=['accuracy', f1_m]
)

history = model_lstm.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/100
832/832 [==============================] - 3s 4ms/step - loss: 0.1444 - accuracy: 0.9620 - f1_m: 0.0010 - val_loss: 0.0916 - val_accuracy: 0.9641 - val_f1_m: 0.0371
Epoch 2/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0773 - accuracy: 0.9699 - f1_m: 0.3477 - val_loss: 0.0676 - val_accuracy: 0.9748 - val_f1_m: 0.5712
Epoch 3/100
832/832 [==============================] - 3s 3ms/step - loss: 0.0581 - accuracy: 0.9787 - f1_m: 0.6349 - val_loss: 0.0552 - val_accuracy: 0.9816 - val_f1_m: 0.7156
Epoch 4/100
832/832 [==============================] - 3s 3ms/step - loss: 0.0492 - accuracy: 0.9819 - f1_m: 0.6994 - val_loss: 0.0485 - val_accuracy: 0.9823 - val_f1_m: 0.7203
Epoch 5/100
832/832 [==============================] - 3s 3ms/step - loss: 0.0454 - accuracy: 0.9827 - f1_m: 0.7165 - val_loss: 0.0478 - val_accuracy: 0.9812 - val_f1_m: 0.6793
Epoch 6/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0424 - accuracy: 0.9838 - f1_m: 0.7380

832/832 [==============================] - 3s 4ms/step - loss: 0.0204 - accuracy: 0.9921 - f1_m: 0.8747 - val_loss: 0.0252 - val_accuracy: 0.9914 - val_f1_m: 0.8709
Epoch 48/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0202 - accuracy: 0.9922 - f1_m: 0.8767 - val_loss: 0.0270 - val_accuracy: 0.9895 - val_f1_m: 0.8424
Epoch 49/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0197 - accuracy: 0.9924 - f1_m: 0.8751 - val_loss: 0.0269 - val_accuracy: 0.9895 - val_f1_m: 0.8436
Epoch 50/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0196 - accuracy: 0.9922 - f1_m: 0.8753 - val_loss: 0.0290 - val_accuracy: 0.9893 - val_f1_m: 0.8226
Epoch 51/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0194 - accuracy: 0.9924 - f1_m: 0.8755 - val_loss: 0.0294 - val_accuracy: 0.9898 - val_f1_m: 0.8508
Epoch 52/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0196 - accuracy: 0.9924 - f1_m: 0.8729 - val_

832/832 [==============================] - 3s 4ms/step - loss: 0.0109 - accuracy: 0.9958 - f1_m: 0.9263 - val_loss: 0.0294 - val_accuracy: 0.9909 - val_f1_m: 0.8572
Epoch 94/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0110 - accuracy: 0.9960 - f1_m: 0.9325 - val_loss: 0.0324 - val_accuracy: 0.9902 - val_f1_m: 0.8549
Epoch 95/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0105 - accuracy: 0.9961 - f1_m: 0.9395 - val_loss: 0.0291 - val_accuracy: 0.9918 - val_f1_m: 0.8722
Epoch 96/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0102 - accuracy: 0.9962 - f1_m: 0.9329 - val_loss: 0.0306 - val_accuracy: 0.9927 - val_f1_m: 0.8898
Epoch 97/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0101 - accuracy: 0.9962 - f1_m: 0.9407 - val_loss: 0.0287 - val_accuracy: 0.9918 - val_f1_m: 0.8761
Epoch 98/100
832/832 [==============================] - 3s 4ms/step - loss: 0.0104 - accuracy: 0.9960 - f1_m: 0.9297 - val_

In [33]:
X_test = enc.transform(X_test_df).toarray().reshape(-1, 4, 20)

In [40]:
y_test=np.around(model_lstm.predict(X_test))

In [41]:
np.savetxt("submission.txt", y_test, fmt="%d")

In [42]:
display(y_test)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)